# Import the Data with Pandas

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("../input/simpsons_dataset.csv") 
data.head()

,raw_character_text,spoken_words
0,Homer Simpson,Never thrown a party? What about that big bash...
1,Homer Simpson,"Bart didn't get one vote?! Oh, this is the wor..."
2,Lisa Simpson,Nothing.
3,Marge Simpson,"Lisa, tell your father."
4,Lisa Simpson,Mr. Bergstrom left today.


In [3]:
data.spoken_words=data.spoken_words.str.replace(r"\?",'¿')
data.head()

,raw_character_text,spoken_words
0,Homer Simpson,Never thrown a party¿ What about that big bash...
1,Homer Simpson,"Bart didn't get one vote¿! Oh, this is the wor..."
2,Lisa Simpson,Nothing.
3,Marge Simpson,"Lisa, tell your father."
4,Lisa Simpson,Mr. Bergstrom left today.


# API-Chat-Emotions Project

### 0. Importing libraries

In [4]:
import requests

### 1. User endpoints

- (GET) `/user/create/<username>`
  - **Purpose:** Create a user and save into DB
  - **Params:** `username` the user name
  - **Returns:** `user_id`

In [5]:
#lista=['Homer Simpson','Marge Simpson','Lisa Simpson','Bart Simpson','Ned Flanders','Moe Szyslak']

In [6]:
list_users=data['raw_character_text'].unique()
list_users

array(['Homer Simpson', 'Lisa Simpson', 'Marge Simpson', 'Bart Simpson'],
      dtype=object)

In [7]:
for x in list_users:
    print(requests.get(f"http://localhost:5000/user/create/{x}").text)

Name already exists
Name already exists
Name already exists
Name already exists


### 2. Chat endpoints:

- (GET) `/chat/create`
  - **Purpose:** Create a conversation to load messages
  - **Params:** An array of users ids `[user_id]`
  - **Returns:** `chat_id`

In [8]:
chat_name="Party"
requests.get(f"http://localhost:5000/chat/create/{chat_name}").text

'Chat already exists'

- (GET) `/chat/<chat_id>/adduser/<user_name>`
  - **Purpose:** Add a user to a chat, this is optional just in case you want to add more users to a chat after it's creation.
  - **Params:** `user_id`
  - **Returns:** `chat_id`

In [9]:
#people=['Homer Simpson','Marge Simpson','Lisa Simpson','Bart Simpson','Ned Flanders','Moe Szyslak']
list_users=data['raw_character_text'].unique()

In [10]:
chat_id=1
for x in list_users:
    print(requests.get(f"http://localhost:5000/chat/{chat_id}/adduser/{x}").text)

Name already exists
Name already exists
Name already exists
Name already exists


- (POST) `/chat/<chat_id>/addmessage`
  - **Purpose:** Add a message to the conversation. Help: Before adding the chat message to the database, check that the incoming user is part of this chat id. If not, raise an exception.
  - **Params:**
    - `chat_id`: Chat to store message
    - `user_id`: the user that writes the message
    - `text`: Message text
  - **Returns:** `message_id`

In [11]:
'''conversation = {"Mmm hors doovers":"Homer Simpson",
                "Homer, please. We've never thrown a party. Now we're gonna pay back all the friends who've invited us to their homes.":"Marge Simpson",
                "But Mom, I want to hear the witty banter of sophisticated adults.":"Lisa Simpson",
                "Yeah, you can't have any fun in bed.":"Bart Simpson",
                "Oh son, when you're older, you'll know better.":"Homer Simpson",
                "Hmmm. Oh, baby! Mmmm. Yeah.":"Homer Simpson",
                "Do we have enough glasses¿":"Marge Simpson",
                "Do we have enough gag ice cubs¿":"Homer Simpson",
                "Homer! Homer! Put a record on!":"Marge Simpson",
                "What are all our friends names again¿":"Homer Simpson",
                "Children! Go!":"Marge Simpson",
                "Hey, anybody mind if I serve as bartender¿ You know, I have a Ph.D in Mixology.":"Ned Flanders",
                "College boy.":"Moe Szyslak",
                "Hey, Homer! Care to try some of my Flanders Planters punch¿":"Ned Flanders",
                "Why not¿ I paid for it.":"Homer Simpson",
                "Hey, Flanders, next time why don't you put a little alcohol in it¿":"Homer Simpson",
                "Au contraire, Simpson! It has three shots of rum, a jigger of bourbon and just little dab-a-roo of creme de cassis for flavor.":"Ned Flanders",
                "Really¿ Well I do have a warm sense of well-being and I seem to be ssslurring my shpeech... you're right! Give me another.":"Homer Simpson",
                "Now, Homer, go easy on the al-ky-hol. Remember last year at the Winfield's party when you threw up in the laundry hamper¿":"Marge Simpson",
                "No.":"Homer Simpson",
               }'''

'conversation = {"Mmm hors doovers":"Homer Simpson",\n                "Homer, please. We\'ve never thrown a party. Now we\'re gonna pay back all the friends who\'ve invited us to their homes.":"Marge Simpson",\n                "But Mom, I want to hear the witty banter of sophisticated adults.":"Lisa Simpson",\n                "Yeah, you can\'t have any fun in bed.":"Bart Simpson",\n                "Oh son, when you\'re older, you\'ll know better.":"Homer Simpson",\n                "Hmmm. Oh, baby! Mmmm. Yeah.":"Homer Simpson",\n                "Do we have enough glasses¿":"Marge Simpson",\n                "Do we have enough gag ice cubs¿":"Homer Simpson",\n                "Homer! Homer! Put a record on!":"Marge Simpson",\n                "What are all our friends names again¿":"Homer Simpson",\n                "Children! Go!":"Marge Simpson",\n                "Hey, anybody mind if I serve as bartender¿ You know, I have a Ph.D in Mixology.":"Ned Flanders",\n                "College boy.

In [12]:
chat_id=1
for x in data['raw_character_text']:
    for y in data['spoken_words']:
        requests.get(f"http://localhost:5000/chat/{chat_id}/mensajes/{y}/adduser/{x}").text

- (GET) `/chat/<chat_id>/list`
  - **Purpose:** Get all messages from `chat_id`
  - **Returns:** json array with all messages from this `chat_id`

In [13]:
numero=1
lista=requests.get(f"http://localhost:5000/chat/{numero}/list").json()[0]['mensajes']
lista[0]

{'autor': 'Homer Simpson',
 'texto': 'Never thrown a party¿ What about that big bash we had with all the champagne and musicians and holy men and everything¿'}

## Sentiment analysis

- (GET) `/chat/<chat_id>/sentiment`
  - **Purpose:** Analyze messages from `chat_id`. Use `NLTK` sentiment analysis package for this task
  - **Returns:** json with all sentiments from messages in the chat
​

In [14]:
requests.get("http://localhost:5000/chat/1/sentiment").json()

{'compound': 1.0, 'neg': 0.123, 'neu': 0.722, 'pos': 0.155}

## NLP Recommender System

- (GET) `/user/<user_id>/recommend`
  - **Purpose:** Recommend friend to this user based on chat contents
  - **Returns:** json array with top 3 similar users

In [15]:
user_name="Homer Simpson"
requests.get(f"http://localhost:5000/user/{user_name}/recommend/1").json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

In [29]:
lista=requests.get(f"http://localhost:5000/chat/{numero}/list").json()[0]['mensajes']

In [30]:
count_vectorizer = CountVectorizer()

In [42]:
data = pd.DataFrame(lista)
data.head()

,autor,texto
0,Homer Simpson,Never thrown a party¿ What about that big bash...
1,Homer Simpson,"Bart didn't get one vote¿! Oh, this is the wor..."
2,Homer Simpson,Nothing.
3,Homer Simpson,"Lisa, tell your father."
4,Homer Simpson,Mr. Bergstrom left today.


In [43]:
sparse_matrix = count_vectorizer.fit_transform(data['texto'])
m = sparse_matrix.todense()

print(m[0])

[[1 0 1 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0
  0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0]]


In [44]:
doc_term_matrix = sparse_matrix.todense()

data_df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=data['autor'])
display(data_df)


,about,ah,all,allowed,allright,alright,although,and,any,anyone,...,wish,with,work,world,worst,would,yeah,yes,you,your
autor,,,,,,,,,,,,,,,,,,,,,
Homer Simpson,1,0,1,0,0,0,0,3,0,0,...,0,1,0,0,0,0,0,0,0,0
Homer Simpson,1,0,0,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
Homer Simpson,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Homer Simpson,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Homer Simpson,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Homer Simpson,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Homer Simpson,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Homer Simpson,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
from sklearn.metrics.pairwise import cosine_similarity as distance

In [46]:
similarity_matrix = distance(data_df,data_df)
similarity_matrix

array([[1.        , 0.11878277, 0.        , ..., 0.        , 0.07856742,
        0.02668803],
       [0.11878277, 1.        , 0.        , ..., 0.        , 0.06299408,
        0.04279605],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.07856742, 0.06299408, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.02668803, 0.04279605, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [47]:
data['autor']

0        Homer Simpson
1        Homer Simpson
2        Homer Simpson
3        Homer Simpson
4        Homer Simpson
             ...      
19705    Homer Simpson
19706    Homer Simpson
19707    Homer Simpson
19708    Homer Simpson
19709    Homer Simpson
Name: autor, Length: 19710, dtype: object

In [ ]:
import seaborn as sns
sim_df = pd.DataFrame(similarity_matrix, columns=data['autor'], index=data['autor'])
display(sim_df)
sns.heatmap(sim_df,annot=True)

autor,Homer Simpson,Homer Simpson,Homer Simpson,Homer Simpson,Homer Simpson,Homer Simpson,Homer Simpson,Homer Simpson,Homer Simpson,Homer Simpson,...,Homer Simpson,Homer Simpson,Homer Simpson,Homer Simpson,Homer Simpson,Homer Simpson,Homer Simpson,Homer Simpson,Homer Simpson,Homer Simpson
autor,,,,,,,,,,,,,,,,,,,,,
Homer Simpson,1.000000,0.118783,0.0,0.0,0.0,0.000000,0.0,0.57735,0.000000,0.000000,...,0.0,0.185185,0.000000,0.116052,0.0,0.172133,0.000000,0.0,0.078567,0.026688
Homer Simpson,0.118783,1.000000,0.0,0.0,0.0,0.154303,0.0,0.00000,0.154303,0.000000,...,0.0,0.207870,0.000000,0.139573,0.0,0.069007,0.109109,0.0,0.062994,0.042796
Homer Simpson,0.000000,0.000000,1.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
Homer Simpson,0.000000,0.000000,0.0,1.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
Homer Simpson,0.000000,0.000000,0.0,0.0,1.0,0.000000,0.0,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Homer Simpson,0.172133,0.069007,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.000000
Homer Simpson,0.000000,0.109109,0.0,0.0,0.0,0.707107,0.0,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.000000
Homer Simpson,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,...,0.0,0.000000,0.235702,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.000000


In [ ]:
import numpy as np
np.fill_diagonal(sim_df.values, 0) # Remove diagonal max values and set those to 0
sim_df.idxmax()

In [ ]:
np.fill_diagonal(sim_df.values, 0) # Remove diagonal max values and set those to 0
sim_df.idxmax()